In [1]:
from mpl_toolkits.mplot3d import axes3d
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from matplotlib.dates import date2num
import pickle
import datetime
import os
!ls ../data/*.pickle

../data/t1_factordata_coreg.pickle    ../data/t1_factordata.pickle
../data/t1_factordata_nocoreg.pickle


In [2]:
pfile1 = open('../data/t1_factordata.pickle', 'r')
pfile2 = open('../data/t1_factordata_coreg.pickle', 'r')
t1_factordata = pickle.load(pfile1)
t1_factordata_coreg = pickle.load(pfile2)

In [3]:
def sig_loss(a, TR, T1):
    from numpy import sin, cos, exp, radians
    a = radians(a)
    l = ((1-cos(a))*exp(-TR/T1))/(1-(cos(a)*exp(-TR/T1)))
    return l

In [4]:
a = 1
TR = np.linspace(1., 200., 1000)
T1 = np.linspace(1., 2000., 1000)
len(np.array(sig_loss(a, TR, T1)))
print(TR[500], T1[999], sig_loss(a, TR[500], T1[999]))
#sig_loss(a, TR, T1)

(100.5995995995996, 2000.0, 0.0029437361408787251)


In [7]:
alpha = [7,10,15,20,30,40]
TR = np.linspace(1., 100., 1000)
T1 = np.linspace(1., 2000., 1000)
num_subplots = len(alpha)
TR, T1 = np.meshgrid(TR, T1)
#fig = plt.figure( figsize=(24,6) )
fig = plt.figure(figsize=plt.figaspect(0.5))
for i,a in enumerate(alpha):
    ax = fig.add_subplot(1 , num_subplots , i+1, projection='3d', axisbg='gray')
    surf = ax.plot_wireframe(TR, T1, sig_loss(a, TR, T1) ,rstride=10, cstride=10, cmap=cm.coolwarm) 
                    #linewidth=0, antialiased=False)
    ax.set_title('alpha = %.0f'%a,fontsize=15)
    ax.set_zlim3d(-0.01, 1.01)
    
#fig.savefig('contours.png', facecolor='grey', edgecolor='none')
#fig.colorbar(surf, shrink=0.5, aspect=10)
plt.show()

In [47]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy

def sig_loss(a, TR, T1):
    from numpy import sin, cos, exp, radians
    a = radians(a)
    l = ((1-cos(a))*exp(-TR/T1))/(1-(cos(a)*exp(-TR/T1)))
    return l

alpha = [7,10,15,20,30,40]
T1_range = numpy.arange(50.0, 2000.0, 1.0).tolist()
TR_range = numpy.arange(1.0, 100.0, 1.0).tolist()
a = alpha[3]
X = T1_range
Y = TR_range
X, Y = numpy.meshgrid(X, Y)
sat = []
for i in range(len(X)):
    sat.append(sig_loss(a, Y[i], X[i]))

Z = sat
fig2 = plt.figure()
az = fig2.gca(projection='3d')
az.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3, cmap=cm.coolwarm)
#cset = az.contour(X, Y, Z, zdir='z', offset=numpy.min(Z)-1, cmap=cm.coolwarm)
cset = az.contour(X, Y, Z, zdir='x', offset=numpy.min(X)-1, cmap=cm.coolwarm)
cset = az.contour(X, Y, Z, zdir='y', offset=numpy.max(Y)+0.05, cmap=cm.coolwarm)
az.set_xlabel('GM <-- T1 --> WM')
az.set_xlim(numpy.min(X)-1, numpy.max(X)+1)
az.set_ylabel('TR')
az.set_ylim(numpy.min(Y)-1, numpy.max(Y)+1)
az.set_zlabel('%saturation')
az.set_zlim(numpy.min(Z)-0.1, numpy.max(Z)+0.1)
az.set_title('Saturation for flip=%.0f Deg.'%a,fontsize=15)
plt.show()

In [1]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy
from numpy import cos, sin, exp, radians, power

def approachSSX(j, a, TR, T1, TE=4.6, T2s=50, M0=1.):
    a = radians(a)
    return M0 * sin(a) * step(a, TR, T1, j) * exp(-TE/T2s)

def step(a, TR, T1, j):
    return (sat(a, TR, T1) + power(cos(a)*exp(-TR/T1), j-1) * (1-sat(a, TR, T1)))

def sat(a, TR, T1):
    return (1-exp(-TR/T1)) / (1-cos(a)*exp(-TR/T1))

def approachSS(j, a, TR, T1, TE, T2s=50, M0=1.):
    a = radians(a)
    return M0 * sin(a) * sub2(a, TR, T1, j) * exp(-TE/T2s)

def sub(a, TR, T1):
    return (1-exp(-TR/T1)) / (1-cos(a)*exp(-TR/T1))

def sub2(a, TR, T1, j):
    return (sub(a, TR, T1) + power(cos(a)*exp(-TR/T1), j-1) * (1-sub(a, TR, T1)))

def sratio(a, TR, T1o, T1m):
    return ((1-exp(-TR/T1m)) * (1-cos(a) * exp(-TR/T1o))) / ((1-cos(a) * exp(-TR/T1m)) * (1-exp(-TR/T1o)))


alpha = [7,10,15,20,30,40]
j_range = numpy.arange(1, 128, 1).tolist()

T1_range = numpy.arange(50.0, 2000.0, 1.0).tolist()
TR_range = numpy.arange(1.0, 100.0, 1.0).tolist()
a = alpha[4]
TR = 14
X = T1_range
Y = j_range
#Y = TR_range
X, Y = numpy.meshgrid(X, Y)
sat = []
for i in range(len(X)):
    sat.append(approachSS(Y[i], a, TR, X[i], TE=4.6, T2s=50, M0=1.))

Z = sat
fig2 = plt.figure()
az = fig2.gca(projection='3d')
az.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3, cmap=cm.coolwarm)
#cset = az.contour(X, Y, Z, zdir='z', offset=numpy.min(Z)-1, cmap=cm.coolwarm)
cset = az.contour(X, Y, Z, zdir='x', offset=numpy.max(X)+0.1, cmap=cm.coolwarm)
cset = az.contour(X, Y, Z, zdir='y', offset=numpy.max(Y)+0.1, cmap=cm.coolwarm)
az.set_xlabel('GM <-- T1 --> WM')
az.set_xlim(numpy.min(X)-1, numpy.max(X)+1)
az.set_ylabel('j value')
az.set_ylim(numpy.min(Y)-1, numpy.max(Y)+1)
az.set_zlabel('normalised signal (M0=1)')
#az.set_zlim(numpy.min(Z)-0.1, numpy.max(Z)+0.1)
az.set_zlim(0,1)
#ax.set_zlim3d(0, 1)
az.set_title('Approach to Steady State for TR=14 and flip=%.0f Deg.'%a,fontsize=15)
plt.show()